In [1]:
import datetime as dt
import pandas as pd

from de.processor.source.dart import DART_FUNDAMENTAL_PROCESSOR, DART_INFO_PROCESSOR
from de.processor.source.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

from de.loader.api_loader.pykrx import get_pykrx_loader

from de.processor.factor.processor import (
    FUNDAMENTAL_FACTOR_PROCESSOR,
    MOVING_AVERAGE_FACTOR_PROCESSOR,
    LINEAR_COEF_FACTOR_PROCESSOR,
    TRADER_FACTOR_PROCESSOR
)

from de.processor.factor.filter import FACTOR_FILTER

In [2]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
        "FILTER": {
            "quantile_conditions": [
                ("CLR", 0.0, 1.0),
                ("TLR", 0.1, 0.7),
                ("NPP", 0.3, 0.8),
                ("OPP", 0.3, 0.7),
                ("TAPP", 0.0, 0.5),
                ("TEPP", 0.0, 0.5),
                ("CAPP", 0.0, 0.4),
                ("CEPP", 0.1, 0.5),
            ]
        },
    },
    "MOVING_AVERAGE": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        },
        "FILTER": {
            "quantile_conditions": [
                ("Close", 0.6, 0.9),
                ("VolumeRotation", 0.0, 0.3),
            ]
        },
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        },
        "FILTER": {
            "quantile_conditions": [
                ("VolumeRotation", 0.4, 0.7),
                ("Close", 0.0, 0.3),
            ],
        },
    },
    "TRADER": {
        "PROCESSOR": {
            "start": (dt.date.today() - dt.timedelta(days=7)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FILTER": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
}

In [3]:
# Load

In [4]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [5]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [7]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [9]:
# Fundamental
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

factor_filter = FACTOR_FILTER(fundamental_factor_df)
f_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FILTER"])

In [10]:
# Technical
## Moving Average
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_factor_df(
    CFG["MOVING_AVERAGE"]["PROCESSOR"]
)

factor_filter = FACTOR_FILTER(moving_average_factors_df)
f_ma_stockcodes = factor_filter.get_filtered_stockcodes(
    CFG["MOVING_AVERAGE"]["FILTER"], f_stockcodes
)

In [11]:
# Technical
## Linear Coef
linear_coef_factor_processor = LINEAR_COEF_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
linear_coef_factors_df = linear_coef_factor_processor.get_factor_df(
    CFG["LINEAR_COEF"]["PROCESSOR"]
)

factor_filter = FACTOR_FILTER(linear_coef_factors_df)
f_lc_stockcodes = factor_filter.get_filtered_stockcodes(CFG["LINEAR_COEF"]["FILTER"], f_stockcodes)

In [15]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)

## Fundamental - Linear Coef - Trader
trader_factor_df = trader_factor_processor.get_factor_df(f_lc_stockcodes, CFG["TRADER"]["PROCESSOR"])
factor_filter = FACTOR_FILTER(trader_factor_df)
f_lc_t_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], f_lc_stockcodes)

## Fundamental - Moving Avergae - Trader
trader_factor_df = trader_factor_processor.get_factor_df(f_ma_stockcodes, CFG["TRADER"]["PROCESSOR"])
factor_filter = FACTOR_FILTER(trader_factor_df)
f_ma_t_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], f_ma_stockcodes)

In [16]:
f_lc_t_stockcodes

{'051390', '058850'}

In [17]:
f_ma_t_stockcodes

{'016380'}